In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Import data

In [2]:
train = pd.read_csv('data/train.csv')
test  = pd.read_csv('data/test.csv')
test_ids = test['PassengerId']

train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Clean data

In [3]:
# find out where information is missing
print(train.isna().sum(axis=0)) 
print( test.isna().sum(axis=0))

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


In [4]:
# Drop everything that is NaN or needs feature engineering because we want it simple (except for that one fare value from test)
train = train.drop(['PassengerId', 'Name', 'Ticket', 'Cabin', 'Age', 'Embarked'], axis=1)
test  =  test.drop(['PassengerId', 'Name', 'Ticket', 'Cabin', 'Age', 'Embarked'], axis=1)
train.head()

,Survived,Pclass,Sex,SibSp,Parch,Fare
0,0,3,male,1,0,7.2500
1,1,1,female,1,0,71.2833
2,1,3,female,0,0,7.9250
3,1,1,female,1,0,53.1000
4,0,3,male,0,0,8.0500


In [5]:
# replace missing fare value from test set by median fare
test.fillna(test['Fare'].median(), inplace=True)
print( test.isna().sum(axis=0))

Pclass    0
Sex       0
SibSp     0
Parch     0
Fare      0
dtype: int64


In [6]:
# convert Sex into numerical values. Both for train and test
LabEn = preprocessing.LabelBinarizer()

train['Sex'] = LabEn.fit_transform(train['Sex'])
test['Sex']  = LabEn.fit_transform( test['Sex'])

train.head()

,Survived,Pclass,Sex,SibSp,Parch,Fare
0,0,3,1,1,0,7.2500
1,1,1,0,1,0,71.2833
2,1,3,0,0,0,7.9250
3,1,1,0,1,0,53.1000
4,0,3,1,0,0,8.0500


# Logistic regression

In [7]:
Y = train['Survived']
X = train.drop(['Survived'], axis=1)

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2)  # split so we can get a rough estimate of the accuracy

In [8]:
classifier_log = LogisticRegression(max_iter=1e4).fit(X_train, Y_train)

In [9]:
predictions_val = classifier_log.predict(X_val)
accuracy_score(Y_val, predictions_val)  # check how good it predicts on the validation set

0.7821229050279329

# Preparation of submission

In [10]:
# compute predictions
predictions_subm = classifier_log.predict(test)

In [11]:
subdf = pd.DataFrame({'PassengerId': test_ids.values, 'Survived': predictions_subm})
subdf.to_csv('data/submission_titanic_v1.csv', index=False)